#### Mount correct folder

In [ ]:
%cd #INSERT FOLDER PATH

#### Import Modules

In [ ]:
from modules.fastec_data_integration import FastecDataIntegrator
from modules.formate_data_integration import FormateDataIntegrator
from modules.rüstmatrix_data_integrator import RüstmatrixDataIntegrator

#### Execute Fastec Feature Integration

In [ ]:
fastec_data_integrator = FastecDataIntegrator(
    raw_data_folder = '00_RawData/',
    raw_data_file_list = [
        'FactMdaState',
        'FactMdaCounter',
        'FactMdaMes',
        'DimWorkcenter',
        'DimShiftEvent',
        'DimMdaOperation',
        'DimMdaCounter',
        'DimMdaState'
    ]
)

fastec_feature_set = fastec_data_integrator.run()

In [ ]:
display(fastec_feature_set)

### External Data Import

##### Formate

In [ ]:
formate_data_integrator = FormateDataIntegrator(
    formate_file_path = '00_ExternalData/Formate mit Produktnummern.xlsx',
    sheet_names = ['FS','PBL','Tuben'],
    fastec_feature_set = fastec_feature_set
)

fastec_formate_dataset = formate_data_integrator.run()

In [ ]:
display(fastec_formate_dataset)

##### Rüstmatrix

In [ ]:
rm_data_integrator = RüstmatrixDataIntegrator(
    formate_file_path = '00_ExternalData/Formate mit Produktnummern.xlsx',
    rüstmatrix_file_path = '00_ExternalData/Rüstmatrixinformationen.xlsx',
    join_feature_set=fastec_formate_dataset
)

final_dataset = rm_data_integrator.run()

#### Filter Dataset for useful orders

In [ ]:
## Remove orders containing "test"
print(f'Dataset Size pre filtering: {len(final_dataset)}')
filtered_final_dataset = final_dataset[~final_dataset['OrderCode'].str.contains('test', case=False, na=False)]
print(f'Dataset Size after removing test orders: {len(filtered_final_dataset)}')
filtered_final_dataset = filtered_final_dataset[
                                        filtered_final_dataset['IsMaintenanceOrder'] == 0
                                    ]
print(f'Dataset Size after removing maintenance orders: {len(filtered_final_dataset)}')
filtered_final_dataset = filtered_final_dataset.query('OEE > 0.01 & OEE <= 1.25')
print(f'Dataset Size after removing invalid OEE orders: {len(filtered_final_dataset)}')


In [ ]:
display(filtered_final_dataset)

In [ ]:
filtered_final_dataset['ProductCode'].sort_values()

#### 

#### Save Integrated and Filtered Dataset

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
filtered_final_dataset.to_parquet(f'01_IntegratedData/{timestamp}_Fastec_Formate_Dataset.parquet')
print(f'Saved Dataset to "01_IntegratedData/{timestamp}_Fastec_Formate_Dataset.parquet"')

### Result Data Overview

In [ ]:
import matplotlib.pyplot as plt
import scienceplots
import pandas as pd

# Use the 'science' style for plots
plt.style.use('science')

In [ ]:
# Applying the threshold to the specified columns
display_features = final_dataset

# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Plotting the histograms with OEE first
axes[0].hist(x=display_features['OEE'], bins=100)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('OEE')

axes[1].hist(x=display_features['QUAL'], bins=100)
axes[1].set_ylabel('Amount of Orders')
axes[1].set_title('Quality Rate')

axes[2].hist(x=display_features['PERF'], bins=100)
axes[2].set_ylabel('Amount of Orders')
axes[2].set_title('Performance Effiency')

axes[3].hist(x=display_features['AVAIL'], bins=100)
axes[3].set_ylabel('Amount of Orders')
axes[3].set_title('Availability Rate')

# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Applying the threshold to the specified columns
display_features = filtered_final_dataset

# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Plotting the histograms with OEE first
axes[0].hist(x=display_features['OEE'], bins=100)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('OEE')

axes[1].hist(x=display_features['QUAL'], bins=100)
#axes[1].set_ylabel('Amount of Orders')
axes[1].set_title('Quality Rate')

axes[2].hist(x=display_features['PERF'], bins=100)
#axes[2].set_ylabel('Amount of Orders')
axes[2].set_title('Performance Effiency')

axes[3].hist(x=display_features['AVAIL'], bins=100)
#axes[3].set_ylabel('Amount of Orders')
axes[3].set_title('Availability Rate')

# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculating mean, variance, and standard deviation for the specified columns
mean_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].mean()
variance_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].var()
std_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].std()

# Creating a DataFrame to store the results
statistics_df = pd.DataFrame({
    'Mean': mean_values,
    'Variance': variance_values,
    'Standard Deviation': std_values
})

# Display the DataFrame
print(statistics_df)

In [ ]:
# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 8, figsize=(16, 4))

# Plotting the histograms with OEE first
axes[0].hist(x=display_features['CALC_PACKGROESSE'], bins=25)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('Package Size in units')

axes[1].hist(x=display_features['PBL_Länge'], bins=25)
axes[1].set_title('Package Leaflet Length')

axes[2].hist(x=display_features['PBL_Breite'], bins=25)
axes[2].set_title('Package Leaflet Width')

axes[3].hist(x=display_features['FS_Länge'], bins=25)
axes[3].set_title('Folding Box Length')

axes[4].hist(x=display_features['FS_Breite'], bins=25)
axes[4].set_title('Folding Box Width')

axes[5].hist(x=display_features['FS_Tiefe'], bins=25)
axes[5].set_title('Folding Box Depth')

axes[6].hist(x=display_features['Tuben_Durchmesser'], bins=25)
axes[6].set_title('Tube Diameter')

axes[7].hist(x=display_features['Tuben_Länge'], bins=25)
axes[7].set_title('Tube Height')

# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
display_features.columns

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Plotting the bar chart for Production Lines
axes[0].bar(display_features['Code'].value_counts().index, display_features['Code'].value_counts().values)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('Total Orders per Production Line')

# Plotting the bar chart with the top 100 products
#top_100_products = display_features['ProductCode'].value_counts().head(100)
#axes[1].bar(top_100_products.index, top_100_products.values)
#axes[1].set_ylabel('Amount of Orders')
#axes[1].set_title('Products')
#
## Hide the x-axis labels (product names)
#axes[1].set_xticklabels([''] * len(top_100_products.index))
#
## Optionally, you can set the x-ticks to have a fixed number of ticks if desired
#axes[1].set_xticks(range(len(top_100_products.index)))

# Plotting the histogram for Order Quantity
axes[1].hist(x=display_features['OrderQuantity'], bins=100)
axes[1].set_ylabel('Amount of Orders')
axes[1].set_title('Order Quantity (units)')

# Calculate the average order size per production line
avg_order_size_per_line = display_features.groupby('Code')['OrderQuantity'].mean()

# Plotting the bar chart for average order size per production line
axes[2].bar(avg_order_size_per_line.index, avg_order_size_per_line.values)
axes[2].set_ylabel('Average Order Quantity (units)')
axes[2].set_title('Avg Order Quantity per Production Line')

# Calculate the average order size per product for the top 100 products
#avg_order_size_per_product = display_features[display_features['ProductCode'].isin(top_100_products.index)] \
#                             .groupby('ProductCode')['OrderQuantity'].mean()

## Plotting the bar chart for average order size per product
#axes[4].bar(avg_order_size_per_product.index, avg_order_size_per_product.values)
#axes[4].set_ylabel('Average Order Size (units)')
#axes[4].set_title('Avg Order Size per Product')
#
## Hide the x-axis labels (product names) for chart 5
#axes[4].set_xticklabels([''] * len(avg_order_size_per_product.index))
#
## Optionally, you can set the x-ticks to have a fixed number of ticks if desired
#axes[4].set_xticks(range(len(avg_order_size_per_product.index)))

# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
display_features['ProductCode'].value_counts()

In [ ]:
# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

# Plotting the bar chart with the top 100 products
top_100_products = display_features['ProductCode'].value_counts().head(150)
axes[0].bar(top_100_products.index, top_100_products.values)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('Top 150 Products by Amount of Orders')

# Hide the x-axis labels (product names)
axes[0].set_xticklabels([''] * len(top_100_products.index))

# Optionally, you can set the x-ticks to have a fixed number of ticks if desired
axes[0].set_xticks(range(len(top_100_products.index)))

# Calculate the average order size per product for the top 150 products
avg_order_size_per_product = display_features[display_features['ProductCode'].isin(top_100_products.index)] \
                             .groupby('ProductCode')['OrderQuantity'].mean()

# Ensure the products are in the same order as in the first chart
avg_order_size_per_product = avg_order_size_per_product.reindex(top_100_products.index)

# Plotting the bar chart for average order size per product, in the same order
axes[1].bar(avg_order_size_per_product.index, avg_order_size_per_product.values)
axes[1].set_ylabel('Average Order Size (units)')
axes[1].set_title('Avg Order Size per Products in Top 150 by Amount of Orders')

# Hide the x-axis labels (product names) for chart 2
axes[1].set_xticklabels([''] * len(avg_order_size_per_product.index))

# Optionally, you can set the x-ticks to have a fixed number of ticks if desired
axes[1].set_xticks(range(len(avg_order_size_per_product.index)))

# Adjust the layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Calculating mean, variance, and standard deviation for the specified columns
mean_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].mean()
variance_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].var()
std_values = filtered_final_dataset[['OEE', 'AVAIL', 'PERF', 'QUAL']].std()

# Creating a DataFrame to store the results
statistics_df = pd.DataFrame({
    'Mean': mean_values,
    'Variance': variance_values,
    'Standard Deviation': std_values
})

# Display the DataFrame
print(statistics_df)


In [ ]:
# Create a figure with subplots in a horizontal line
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Plotting the histograms with OEE first
axes[0].hist(x=display_features['PBT']/3600000, bins=50)
axes[0].set_ylabel('Amount of Orders')
axes[0].set_title('Planned Production Time (hrs)')

axes[1].hist(x=display_features['DT']/3600000, bins=50)
axes[1].set_title('Downtime (hrs)')

axes[2].hist(x=display_features['APT']/3600000, bins=50)
axes[2].set_title('Operating Time (hrs)')



#### Feature Creation Charts

In [ ]:
import matplotlib.pyplot as plt
import scienceplots
import pandas as pd
import seaborn as sns

# Use the 'science' style for plots
plt.style.use('science')

In [ ]:
from modules.utils import load_latest_dataset_from_storage

data_ml_integration = load_latest_dataset_from_storage('01_IntegratedData', 'Fastec_Formate_Dataset')

In [ ]:
# Feature Engineering: Create a unique identifier for each product changeover
data_ml_integration['ProductChange'] = data_ml_integration['Previous_ProductCode'] + "-" + data_ml_integration['ProductCode']
data_ml_integration['Auftragswechsel'] = data_ml_integration['Auftragswechsel']/3600000

print(f'Length before dropping NaNs:{len(data_ml_integration)}')
data_ml_integration = data_ml_integration.dropna(subset=['Previous_ProductCode'])
data_ml_integration = data_ml_integration[data_ml_integration['Previous_ProductCode'].str.strip() != '']
data_ml_integration = data_ml_integration.dropna(subset=['ProductChange', 'Auftragswechsel'])
print(f'Length after dropping NaNs:{len(data_ml_integration)}')
#data_ml_integration.loc[data_ml_integration['Auftragswechsel']  > threshold, 'Auftragswechsel'] = threshold
# Ensure 'ProductChange' is a string and 'Auftragswechsel' is numeric
data_ml_integration['ProductChange'] = data_ml_integration['ProductChange'].astype(str)

In [ ]:
# Extract Percentiles
# Group by 'ProductChange' and calculate the percentiles for 'Auftragswechsel'
percentiles_df = data_ml_integration.groupby(['ProductChange'])['Auftragswechsel'].quantile([0, 0.1, 0.25]).unstack()

# Rename the columns to reflect the percentile values
percentiles_df.columns = ['0%_Auftragswechsel', '10%_Auftragswechsel', '25%_Auftragswechsel']

# Reset the index so 'ProductChange' becomes a column
percentiles_df.reset_index(inplace=True)

# Convert 'ProductChange' to string
percentiles_df['ProductChange'] = percentiles_df['ProductChange'].astype(str)

# Adjusted plot for the percentiles using scatter plot
plt.figure(figsize=(10, 6))
# Transforming the DataFrame for easier plotting
percentile_long_df = percentiles_df.melt(id_vars='ProductChange', var_name='Percentile', value_name='Time')
sns.scatterplot(data=percentile_long_df, x='ProductChange', y='Time', hue='Percentile', style='Percentile', s=100)
plt.xticks([], [])  # Only remove labels, keep ticks if necessary
plt.title('Percentile Values of Auftragswechsel by ProductChange')
plt.tight_layout()
plt.show()

# Adjusted plot for the original distribution of Auftragswechsel using bubble chart
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_ml_integration, x='ProductChange', y='Auftragswechsel', legend=False, sizes=(20, 200))
plt.title('Original Distribution of Auftragswechsel as Bubble Chart')
plt.xticks([], [])  # Only remove labels, keep ticks if necessary
plt.xlabel('ProductChange')
plt.ylabel('Auftragswechsel Time')
plt.tight_layout()
plt.show()